## 최근 데이터 10개로 predict 해보기

In [1]:
import pandas as pd
import numpy as np

In [2]:
from bs4 import BeautifulSoup
import requests
import time

In [3]:
url_daegu = 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=%EB%8C%80%EA%B5%AC&sort=0&photo=0&field=0&pd=0&ds=&de=&cluster_rank=47&mynews=0&office_type=0&office_section_code=0&news_office_checked=&office_category=0&service_area=0&nso=so:r,p:all,a:all&start=1'
response = requests.get(url_daegu)

In [4]:
html_ = BeautifulSoup(response.text)
newsList = html_.find('div', class_ = 'group_news' ).find_all('div', class_ = 'news_info')

In [5]:
newsList[0].find_all('a')[-1]['href']

'https://n.news.naver.com/mnews/article/003/0012251555?sid=101'

In [6]:
url_ = newsList[0].find_all('a')[-1]['href']
response_ = requests.get(url_)

In [7]:
html__ = BeautifulSoup(response_.text)

In [8]:
title = html__.find('div',class_= 'media_end_head_title').text.replace('\n','').replace('\t','').replace('\r','')
title

'"서울부터 대구까지 232㎞ 거리"…22만개 팔린 \'이 소파\''

In [9]:
content = html__.find('article',id = 'dic_area').text.replace('\n','').replace('\t','').replace('\r','')
content

"신세계까사 까사미아 캄포 소파패브릭·착석감, 모듈기능이 강점[서울=뉴시스] 신세계까사 까사미아 캄포 소파. (사진=신세계까사 제공) 2023.12.06. photo@newsis.com[서울=뉴시스] 배민욱 기자 = 신세계까사의 베스트셀러 소파 '캄포'가 누적 판매 22만개 돌파했다. 6일 신세계까사에 따르면 캄포는 신세계까사가 신세계그룹 편입 이후 대표 브랜드 '까사미아'의 상품 경쟁력 강화 일환으로 선보인 소파다. 2019년 출시 당시 가죽 소파 선호도가 높던 국내 시장에서 10분에 1개씩 팔리며 패브릭 소파 열풍을 주도했다. 판매된 캄포 소파 제품을 일렬로 나열하면 232㎞다. 서울에서 대구까지 갈 수 있는 거리다.캄포는 4년째 베스트셀러의 자리를 지키고 있다. 캄포 소파의 인기 배경에는 기능성 패브릭, 편안한 착석감 , 모듈 기능의 장점이 고르게 작용된 것으로 보인다. 신세계까사는 캄포 누적 판매 22만개 돌파를 기념해 '어메이징 캄포' 프로모션을 내년 1월 14일까지 진행한다. 지난달 30일 이전까지 신세계까사의 온라인 쇼핑 플랫폼 '굳닷컴'과 기타 온라인몰, 까사미아 오프라인 매장에서 캄포를 구매한 고객은 '굳닷컴'에서 캄포 구매 인증 시 쇼핑 지원금 1만 포인트를 받는다. 굳닷컴이나 까사미아 오프라인 매장에서 캄포를 구매한 고객이 매장에서 직원을 통해 구매 인증 후 까사미아 제품을 구매하면 금액별로 신세계상품권 2만·4만·6만원권도 선물한다.이벤트 기간 내 굳닷컴에서 캄포 클래식 또는 슬림 소파 신규 구매 고객은 기본 모듈 커버 1세트와 굳포인트 5만점 페이백 등의 혜택을 제공 받는다. 17일까지 매장에서 웨딩·입주 클럽 가입 후 캄포 플러스 제품을 300만원 이상 구매하면 신세계상품권 10만원권이 제공된다. SNS(사회관계망서비스) 공유 이벤트도 있다. 캄포와 함께 한 일상의 순간을 개인 SNS에 공유하면 추첨을 통해 까사미아 상품권 50만원, 굳포인트 3만점, 스타벅스 기프티콘 등이 증정된다. 자세한 내용은 까사미아 공식 인스타그램에서 확인할 

### 신뢰도 모델

In [10]:
# 토큰화, 벡터화
from keras import preprocessing
from keras.utils import pad_sequences
from keras.models import load_model

def reliability_input(title, content):
    vocab_size = 1000            # 상위 1000개의 고유한 단어로 제한한다는 의미
    max_sequence_length = 100    # 시퀀스에 허용되는 최대 토큰 수(단어,문자  단위)
    
    # num_words에 지정된 만큼만 숫자로 반환 - 부족하면 0으로 채우고, 넘치면 잘림
    tokenizer = preprocessing.text.Tokenizer(num_words=vocab_size, oov_token="<oov>")    # oov_tok을 사용하여 사전에 없는 단어집합을 만듬
    tokenizer.fit_on_texts([title])
    title_sequences = tokenizer.texts_to_sequences([title])
    title_padded = pad_sequences(title_sequences, maxlen=max_sequence_length)
    tokenizer.fit_on_texts([content])
    content_sequences = tokenizer.texts_to_sequences([content])
    content_padded = pad_sequences(content_sequences, maxlen=max_sequence_length)


    # Load the saved model
    loaded_model = load_model("best_cnn_model.h5")
    # Make predictions
    predictions = loaded_model.predict([title_padded, content_padded])
    predictions = predictions + 15
    predictions = np.clip(predictions, a_min=0,a_max=100)
    predictions = int(predictions.round())

    return predictions

In [8]:
title

'"서울부터 대구까지 232㎞ 거리"…22만개 팔린 \'이 소파\''

In [9]:
content

"신세계까사 까사미아 캄포 소파패브릭·착석감, 모듈기능이 강점[서울=뉴시스] 신세계까사 까사미아 캄포 소파. (사진=신세계까사 제공) 2023.12.06. photo@newsis.com[서울=뉴시스] 배민욱 기자 = 신세계까사의 베스트셀러 소파 '캄포'가 누적 판매 22만개 돌파했다. 6일 신세계까사에 따르면 캄포는 신세계까사가 신세계그룹 편입 이후 대표 브랜드 '까사미아'의 상품 경쟁력 강화 일환으로 선보인 소파다. 2019년 출시 당시 가죽 소파 선호도가 높던 국내 시장에서 10분에 1개씩 팔리며 패브릭 소파 열풍을 주도했다. 판매된 캄포 소파 제품을 일렬로 나열하면 232㎞다. 서울에서 대구까지 갈 수 있는 거리다.캄포는 4년째 베스트셀러의 자리를 지키고 있다. 캄포 소파의 인기 배경에는 기능성 패브릭, 편안한 착석감 , 모듈 기능의 장점이 고르게 작용된 것으로 보인다. 신세계까사는 캄포 누적 판매 22만개 돌파를 기념해 '어메이징 캄포' 프로모션을 내년 1월 14일까지 진행한다. 지난달 30일 이전까지 신세계까사의 온라인 쇼핑 플랫폼 '굳닷컴'과 기타 온라인몰, 까사미아 오프라인 매장에서 캄포를 구매한 고객은 '굳닷컴'에서 캄포 구매 인증 시 쇼핑 지원금 1만 포인트를 받는다. 굳닷컴이나 까사미아 오프라인 매장에서 캄포를 구매한 고객이 매장에서 직원을 통해 구매 인증 후 까사미아 제품을 구매하면 금액별로 신세계상품권 2만·4만·6만원권도 선물한다.이벤트 기간 내 굳닷컴에서 캄포 클래식 또는 슬림 소파 신규 구매 고객은 기본 모듈 커버 1세트와 굳포인트 5만점 페이백 등의 혜택을 제공 받는다. 17일까지 매장에서 웨딩·입주 클럽 가입 후 캄포 플러스 제품을 300만원 이상 구매하면 신세계상품권 10만원권이 제공된다. SNS(사회관계망서비스) 공유 이벤트도 있다. 캄포와 함께 한 일상의 순간을 개인 SNS에 공유하면 추첨을 통해 까사미아 상품권 50만원, 굳포인트 3만점, 스타벅스 기프티콘 등이 증정된다. 자세한 내용은 까사미아 공식 인스타그램에서 확인할 

In [11]:
pred_=reliability_input(title, content)
pred_

1/1 [==============================] - 1s 508ms/step


63

In [12]:
ds = pd.read_csv('../DB_datas/output.csv')
ds = ds[['제목','본문']]
ds

,제목,본문
0,"하윤기, 덩크 9방에 생애 첫 올스타전 MVP",팀 이대성 122 117로 팀 허웅 제압 전성현 3점슛 9개 분전허웅 생애 첫 ...
1,"장제원, 나경원 향해 “제2의 유승민 되지 말기를”",제2의 진박감별사가 당 쥐락펴락 나경원 비판에 응수 국민의힘 장제원 의원 나...
2,[대구 국회의원 평가 여론조사] 북구갑 양금희 당직 맡아 국회평가 높아,양금희 국민의힘 의원 대구 북구갑 . 대구...
3,[대구 국회의원 평가 여론조사] 서구 김상훈 3선 의정활동 평가 무난,김상훈 국민의힘 의원 대구 서구 . 대구 ...
4,[대구 국회의원 평가 여론조사] 동구갑 류성걸 의원,류성걸 국민의힘 의원 대구 동구갑 . 류성...
...,...,...
39941,1년 전 銀을 金으로 우상혁 '위대한 도전',우상혁이 20일 2023 부다페스트 세계육상선수권 남자 높이뛰기 예선에서 바를 넘고...
39942,[인사] 중소벤처기업부 외,중소벤처기업부 실장급 전보 중소기업정책실장 이대희 소상공인정책실장 원영준 국...
39943,"'명예훼손 실형' 김용호, 조국 김건모 전처에 사과 ""깊이 반성""",유튜버 김용호씨. 유튜브 채널 강용석 나이트 라이브 캡처연예 출신 유튜...
39944,이러니 치안 불안할 수 밖에 현장 누빌 순경 1.4만명 부족,윤희근 경찰청장이 18일 충북 충주시 중앙경찰학교에서 열린 신임경찰 제312기 졸업...
